In [ ]:
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import requests

In [ ]:
course_name = 'machine-learning-zoomcamp'

In [ ]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'

def get_json(endpoint):
  docs_url = f'{base_url}/{endpoint}?raw=1'
  docs_response = requests.get(docs_url)
  return docs_response.json()

def get_csv(endpoint):
  docs_url = f'{base_url}/{endpoint}?raw=1'
  return pd.read_csv(ground_truth_url)

# Question 1

In [23]:
model_name = 'multi-qa-distilbert-cos-v1'
user_question = "I just discovered the course. Can I still join it?"

embedding_model = SentenceTransformer(model_name)
embedding_vector = embedding_model.encode(user_question)

In [ ]:
embedding_vector[0]

# Question 2

In [9]:
documents = get_json('03-vector-search/eval/documents-with-ids.json')

def filterCourse(q):
    return q['course'] == course_name

filtered_documents = list(filter(filterCourse, documents))
len(filtered_documents)

In [26]:
embeddings = []

for document in filtered_documents:
    qa_text = f"{document['question']} {document['text']}"
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

In [ ]:
X = np.array(embeddings)

In [ ]:
X.shape

# Question 3

In [29]:
scores = X.dot(embedding_vector)

In [ ]:
max(scores)

# Question 4

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

## usage
# search_engine = VectorSearchEngine(documents=documents, embeddings=X)
# search_engine.search(embedding_vector, num_results=5)

In [ ]:
df_ground_truth = get_csv('03-vector-search/eval/ground-truth-data.csv')
df_ground_truth = df_ground_truth[df_ground_truth.course == course_name]
ground_truth = df_ground_truth.to_dict(orient='records')

search_engine = VectorSearchEngine(documents=ground_truth, embeddings=X)

In [51]:
results = [];

for record in ground_truth:
    query = record['question']
    document_id = record['document']
    v_query = embedding_model.encode(query)
    current_results = search_engine.search(v_query, num_results=5)
    results.append(current_results)

In [ ]:
hits = 0

for arr in results:
  if document_id in result['id']:
    hits += 1
    break

In [ ]:
print('hits: ' + hits)
print('len(ground_truth): ' + len(ground_truth))
print('hitrate: ' + hits / len(ground_truth))

# Question 5

In [ ]:
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
index_name = "course-questions"

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

In [ ]:
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in tqdm(ground_truth):
    es_client.index(index=index_name, document=doc)

In [ ]:
response = es_client.search(index=index_name, body=user_question)

In [ ]:
response

# Question 6